# Cleaning the dates and times
This script will explain how I cleaned the dates and times of the Monroe County crash data. It will explain the logic that went into the cleaning scripts `clean_datetime.py` and `clean_times.py` in the `cleaning-workflow/cleaning-scripts` folder.

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import date, time

pd.set_option('display.max_columns', None)

## Standardizing date/time columns
The goal is to create a `DateTime` column which is standardized across all of the data files. This will require looking at how the date/time data is structured in each source file, and parsing out strange formats before being able to create this finalized column.

So let's see what weird date/time formatting we'll be working with.

In [3]:
# starting with the 2022 data
crash_df = pd.read_csv('../data/source-data/moco-crash-2021.csv')  
crash_df[['Collision Date','Collision Time']]

,Collision Date,Collision Time
0,2021-12-31T00:00:00,12:30 AM
1,2021-12-31T00:00:00,12:50 AM
2,2021-12-31T00:00:00,12:31 PM
3,2021-12-31T00:00:00,10:15 PM
4,2021-12-30T00:00:00,2:41 AM
...,...,...
3052,2021-01-01T00:00:00,7:59 AM
3053,2021-01-01T00:00:00,9:40 AM
3054,2021-01-01T00:00:00,9:55 AM
3055,2021-01-01T00:00:00,10:35 AM


In [14]:
def format_search(re_string,string):
    return bool(re.search(re_string, string))

In [16]:
# how many of the Collision Date entries follow this format?
re_search_string = '[\d]{4}-[\d]{2}-[\d]{2}T[\d]{2}:[\d]{2}:[\d]{2}'
crash_df['Collision Date'].apply(lambda x: format_search(re_search_string,x)).value_counts()

True    3057
Name: Collision Date, dtype: int64

Looks like every `Collision Date` entry follows the exact same format! Yay. Now let's check the `Collision Time`. 

In [17]:
re_search_string = '[\d]{1,2}:[\d]{2} [AP]M'
crash_df['Collision Time'].apply(lambda x: format_search(re_search_string,x)).value_counts()

True    3057
Name: Collision Time, dtype: int64

Same for `Collision Time`. Knowing the format is consistent, we can now apply a simple cleaning function to create a combined `DateTime` field.

In [20]:
# given a date string (`Collision Date` field) and a time string (`Collision Time` field),
def get_datetime(date_string, time_string):
    time_string = str(time_string)
    # combine the date and time into one `datetime` object
    date_val = date(pd.to_datetime(date_string).year,
                 pd.to_datetime(date_string).month,
                 pd.to_datetime(date_string).day
                 )
    if time_string:
        # if the time string is `NaN`, set it to midnight
        try:
            time_val = time(pd.to_datetime(time_string).hour,
                      pd.to_datetime(time_string).minute
                     )
        except:
            time_val = time(0,0,0)
        
    return(pd.Timestamp.combine(date_val,time_val))
                            

In [30]:
crash_df['DateTime'] = crash_df.apply(lambda row: get_datetime(row['Collision Date'], row['Collision Time']), axis=1)
crash_df['DateTime']

0      2021-12-31 00:30:00
1      2021-12-31 00:50:00
2      2021-12-31 12:31:00
3      2021-12-31 22:15:00
4      2021-12-30 02:41:00
               ...        
3052   2021-01-01 07:59:00
3053   2021-01-01 09:40:00
3054   2021-01-01 09:55:00
3055   2021-01-01 10:35:00
3056   2021-01-01 23:21:00
Name: DateTime, Length: 3057, dtype: datetime64[ns]

## More complex/weird date/time formats
The first few `csv`s had simple date/time formats, but the older data is trickier. Let's figure out how to clean this.

### 2013-2018 data

In [4]:
# starting with the 2013-2018 data
crash_df_1318 = pd.read_csv('../data/source-data/moco-crash-2013-2018.csv', encoding='unicode_escape')  
crash_df_1318[['DATE', 'TIME']]

,DATE,TIME
0,7/30/2014,1:16 AM
1,5/11/2015,9:50 AM
2,1/4/2016,2:06 PM
3,6/14/2016,3:00 PM
4,8/6/2013,1340
...,...,...
22406,11/16/2018,5:58 PM
22407,12/12/2015,3:21 AM
22408,11/6/2017,7:50 AM
22409,6/16/2016,7:28 PM


Let's check the date/time formats.

In [38]:
# how many of the DATE entries follow the #/#/#### format?
re_search_string = '[\d]{1,2}\/[\d]{1,2}\/[\d]{4}'
crash_df_1318['DATE'].apply(lambda x: format_search(re_search_string,x)).value_counts()

True    22411
Name: DATE, dtype: int64

In [39]:
# how many of the TIME entries follow the #:## AM format?
re_search_string = '[\d]{1,2}:[\d]{2} [AP]M'
crash_df_1318['TIME'].apply(lambda x: format_search(re_search_string,x)).value_counts()

True     19350
False     3061
Name: TIME, dtype: int64

Uh-oh, > 13% of entries don't follow that format. Let's see what the other formats are.

In [52]:
crash_df_1318['flag'] = crash_df_1318['TIME'].apply(lambda x: format_search(re_search_string,x))
crash_df_1318[crash_df_1318['flag'] == False]['TIME'].sample(10)

4732      853
12527    2225
10329    2047
3053     1730
11356    1713
2468     1930
6245     2341
12831    2232
4022     2200
10402    2107
Name: TIME, dtype: object

In [54]:
# now drop the flag col
crash_df_1318 = crash_df_1318.drop(columns=['flag'])

It appears that most of them are either 3 or 4 digit numbers. For cleaning purposes, I will assume that a three-digit number like `853` means `8:53 AM`, and a four-digit number like `2341` means `11:41 PM` (military time). 

Let's check if that covers the remaining formats, and then make a new cleaning script to deal with this variation.

In [65]:
# how many of the TIME entries follow the #:## AM format?
re_search_string = '[\d]{3,4}'
crash_df_1318['TIME'].apply(lambda x: format_search(re_search_string,x)).value_counts()

False    19410
True      3001
Name: TIME, dtype: int64

Looks like this takes care of most of the 3061 values that didn't match the first format. Let's go ahead with the plan of cleaning these like military time.

In [67]:
# Function to clean the times
def clean_military_time(Time):
    
    # Search for 3- or 4-digit times
    # these are in military time. so need to be converted to regular time to match the rest.
    if re.search('\d{3,4}', Time):
        
        # ex 1230, 1024
        if len(Time) == 4:
            if int(Time[0:2])>12:
                return str(int(Time[0:2]) - 12) + ":" + Time[2:4] + " PM"
            elif Time[0:2] == '12':
                return Time[0:2] + ":" + Time[2:4] + " PM"
            else:
                return Time[0:2] + ":" + Time[2:4] + " AM"
            
        # ex 130, 930, 725
        if len(Time) == 3:
            if Time[0:1] == '0':
                return "12:" + Time[1:3] + " AM"
            else:
                return Time[0:1] + ":" + Time[1:3] + " AM" 
        return Time
    else:
        # if clean up not needed, return the same name
        return Time

In [68]:
# remove all times with one or two digit numbers 
def wrong_times_to_nan(Time):
    if not re.search('\d{1,2}:\d{2} [A,P]M', Time):
        return float('NaN')
    else:
        return Time

In [69]:
crash_df_1318['TIME'] = crash_df_1318['TIME'].apply(clean_military_time)
crash_df_1318['TIME'] = crash_df_1318['TIME'].apply(wrong_times_to_nan)

In [72]:
# how many follow the am/pm format now?
re_search_string = '[\d]{1,2}:[\d]{2} [AP]M'
crash_df_1318['TIME'].apply(lambda x: format_search(re_search_string,str(x))).value_counts()

True     22351
False       60
Name: TIME, dtype: int64

Now the vast majority are cleaned. This allows us to run the `DateTime` formula from above, now that the time/date values are standardized.

In [73]:
crash_df_1318['DateTime'] = crash_df_1318.apply(lambda row: get_datetime(row['DATE'], row['TIME']), axis=1)
crash_df_1318['DateTime']

0       2014-07-30 01:16:00
1       2015-05-11 09:50:00
2       2016-01-04 14:06:00
3       2016-06-14 15:00:00
4       2013-08-06 13:40:00
                ...        
22406   2018-11-16 17:58:00
22407   2015-12-12 03:21:00
22408   2017-11-06 07:50:00
22409   2016-06-16 19:28:00
22410   2015-12-19 01:44:00
Name: DateTime, Length: 22411, dtype: datetime64[ns]

### 2003-2015 data

In [5]:
# starting with the 2013-2018 data
crash_df_0315 = pd.read_csv('../data/source-data/moco-crash-2003-2015.csv', encoding='unicode_escape')  
crash_df_0315[['Year','Month','Day','Hour']]

,Year,Month,Day,Hour
0,2015,1,5,0.0
1,2015,1,6,1500.0
2,2015,1,6,2300.0
3,2015,1,7,900.0
4,2015,1,7,1100.0
...,...,...,...,...
53938,2003,10,6,1700.0
53939,2003,11,3,800.0
53940,2003,12,5,1200.0
53941,2003,12,1,700.0


The 2003-2015 data uses yet another different format, logging each part of the date/time in its own column. This won't be too hard to convert into a date, but the `Hour` format will need its own custom cleaning function to make sense as a real time.

In [81]:
def is_float(val):
    return isinstance(val, float)

In [83]:
# are all the `Hour` values floats?
crash_df_0315['Hour'].apply(is_float).value_counts()

True    53943
Name: Hour, dtype: int64

In [86]:
def clean_float_times(hour_string):
    # test if float exists
    if hour_string == hour_string:
        return str(time(int(hour_string / 100)))

In [90]:
crash_df_0315['Collision Time'] = crash_df_0315['Hour'].apply(clean_float_times)

In [91]:
# did this work for most values? 
# how many follow the am/pm format now?
re_search_string = '[\d]{1,2}:[\d]{2}:00'
crash_df_0315['Collision Time'].apply(lambda x: format_search(re_search_string,str(x))).value_counts()

True     53718
False      225
Name: Collision Time, dtype: int64

That took care of most of the times. Now, let's form a `DateTime` column just like the others.

In [104]:
crash_df_0315['Collision Date'] = pd.to_datetime(crash_df_0315[['Year', 'Month', 'Day']])

In [106]:
crash_df_0315['DateTime'] = crash_df_0315.apply(lambda x: get_datetime(x['Collision Date'], x['Collision Time']), axis=1)

In [110]:
# now drop the cols that were used to create datetime, since they're no longer useful
crash_df_0315 = crash_df_0315.drop(columns=["Year","Month","Day","Weekend?","Hour","Collision Time","Collision Date"])

In [111]:
crash_df_0315

,Master Record Number,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,DateTime
0,902363382,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,2015-01-05 00:00:00
1,902364268,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,2015-01-06 15:00:00
2,902364412,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,2015-01-06 23:00:00
3,902364551,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,2015-01-07 09:00:00
4,902364615,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,2015-01-07 11:00:00
...,...,...,...,...,...,...,...,...
53938,900084526,2-Car,No injury/unknown,IMPROPER LANE USAGE,DUNN & WHITE LOT WEST,0.000000,0.000000,2003-10-06 17:00:00
53939,900089213,1-Car,No injury/unknown,UNSAFE SPEED,RED OAK & SR446,0.000000,0.000000,2003-11-03 08:00:00
53940,900095322,2-Car,No injury/unknown,BRAKE FAILURE OR DEFECTIVE,2ND ST & WALNUT,0.000000,0.000000,2003-12-05 12:00:00
53941,900099922,2-Car,No injury/unknown,UNSAFE BACKING,NINETH & NORTH,0.000000,0.000000,2003-12-01 07:00:00


## The end
Now the `DateTime` field is consistent across all the source data, and can be used for mapping and other analysis more easily.
